# Recurrent Neural Network with online BPTT

This notebook is based on [DennyBritz's RNN Tutorial](https://github.com/dennybritz/nn-from-scratch). 

The main change is that training is done in an online way, which means weights are updated after every time steps rather than only once at the end of a sequence. In other words, Forward and Backward have fine grain coupling, running alternatively.

In [138]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [139]:
import os
%cd /content/drive/MyDrive/graduation_project/FPTT-on-ANN/RNN/rnn-tutorial-rnnlm-py3/
!pwd

/content/drive/MyDrive/graduation_project/FPTT-on-ANN/RNN/rnn-tutorial-rnnlm-py3
/content/drive/MyDrive/graduation_project/FPTT-on-ANN/RNN/rnn-tutorial-rnnlm-py3


In [140]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime
from utils import *
#!pip3 install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [141]:
# Download NLTK model data (you need to do this once)
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Downloadi

True

In [142]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print("Reading CSV file...")
with open('data/reddit-comments-2015-08.csv', 'rt', encoding='utf8') as f:
    reader = csv.reader(f, skipinitialspace=True)
    next(reader)
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print("Parsed %d sentences." % (len(sentences)))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print("\nExample sentence: '%s'" % sentences[0])
print("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

Reading CSV file...
Parsed 79184 sentences.
Found 63023 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'appointments' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'


In [143]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

<ipython-input-143-4547043b0b3a>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
<ipython-input-143-4547043b0b3a>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


Here's an actual training example from our text:

In [144]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print("x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example))

x:
SENTENCE_START what are n't you understanding about this ? !
[0, 52, 28, 17, 10, 858, 55, 26, 35, 70]

y:
what are n't you understanding about this ? ! SENTENCE_END
[52, 28, 17, 10, 858, 55, 26, 35, 70, 1]


In [145]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        

#### Forward Propagation

Next, let's implement the forward propagation (predicting word probabilities) defined by our equations above:

In [146]:
def softmax(z): return np.exp(z)/((np.exp(z)).sum())

In [147]:
def forward_propagation(self, x):
    # The total number of time steps
    T = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    # The outputs at each time step. Again, we save them for later.
    o = np.zeros((T, self.word_dim))
    # For each time step...
    for t in np.arange(T):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

RNNNumpy.forward_propagation = forward_propagation

In [148]:
def predict(self, x):
    # Perform forward propagation and return index of the highest score
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)

RNNNumpy.predict = predict

In [149]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print(o.shape)
print(o)

(45, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012566 0.00012567 0.0001254  ... 0.00012563 0.00012532 0.00012528]
 [0.00012581 0.00012334 0.00012526 ... 0.0001256  0.00012492 0.00012513]
 ...
 [0.00012441 0.00012512 0.0001248  ... 0.00012496 0.00012448 0.000126  ]
 [0.00012493 0.00012393 0.00012497 ... 0.00012428 0.00012527 0.00012465]
 [0.00012493 0.00012557 0.00012502 ... 0.00012481 0.00012429 0.00012561]]


In [150]:
predictions = model.predict(X_train[10])
print(predictions.shape)
print(predictions)

(45,)
[1284  397 2044 5314 3865 1042 7598 6200 1041 1042 7598 3106 6892 4941
 4480 5370 5638 4591 5407 2314 2798 2887  903 4719 7051 5151   18 4223
 6127 1499 1207 1814 7522 4911 4545   35 3528 2314  794 1293 1305 1692
  828 2874 7766]


#### Calculating the Loss

To train our network we need a way to measure the errors it makes. We call this the loss function $L$, and our goal is find the parameters $U,V$ and $W$ that minimize the loss function for our training data. A common choice for the loss function is the [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression). If we have $N$ training examples (words in our text) and $C$ classes (the size of our vocabulary) then the loss with respect to our predictions $o$ and the true labels $y$ is given by:

$
\begin{aligned}
L(y,o) = - \frac{1}{N} \sum_{n \in N} y_{n} \log o_{n}
\end{aligned}
$

The formula looks a bit complicated, but all it really does is sum over our training examples and add to the loss based on how off our prediction are. The further away $y$ (the correct words) and $o$ (our predictions), the greater the loss will be. We implement the function `calculate_loss`:

In [151]:
def calculate_total_loss(self, x, y):
    L = 0
    # For each sentence...
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        # We only care about our prediction of the "correct" words
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        # Add to the loss based on how off we were
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x,y)/N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

Let's take a step back and think about what the loss should be for random predictions. That will give us a baseline and make sure our implementation is correct. We have $C$ words in our vocabulary, so each word should be (on average) predicted with probability $1/C$, which would yield a loss of $L = -\frac{1}{N} N \log\frac{1}{C} = \log C$:

In [152]:
# Limit to 1000 examples to save time
print("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print("Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000]))

<ipython-input-151-3f0be9da26f3>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


Expected Loss for random predictions: 8.987197
Actual loss: 8.987374


Pretty close! Keep in mind that evaluating the loss on the full dataset is an expensive operation and can take hours if you have a lot of data!

#### Training the RNN with SGD and Backpropagation Through Time (BPTT)


In [153]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T): #[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print("Backpropagation step t=%d bptt step=%d " % (t, bptt_step))
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

In [154]:
# online BPTT that updates weights after each time step
# to be specific, for each time step
# 1. forward
# 2. backward
# 3. update
# bptt_truncate being 0 -- no look back

def bptt_online_learning(self, x, y, learning_rate):
    T = len(y)

    o = np.zeros((T, self.word_dim))
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    delta_o = o
    
    for t in np.arange(T):

        # Perform forward propagation for one time step
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))

        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)

        delta_o[t, y[t]] -= 1.
        
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))

        # Backpropagation through time (for at most self.bptt_truncate steps)
        # for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
        for bptt_step in np.arange(max(0, t), t+1)[::-1]:
            # print("Backpropagation step t=%d bptt step=%d " % (t, bptt_step))
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        
        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW 

RNNNumpy.bptt_online_learning = bptt_online_learning

In [155]:
# fptt training
# based on bptt online training
# further introduce running mean, running estimate (lambda) 

def fptt_training(self, x, y, learning_rate):
    T = len(y)

    alpha = 0.1

    o = np.zeros((T, self.word_dim))
    s = np.zeros((T + 1, self.hidden_dim))
    s[-1] = np.zeros(self.hidden_dim)
    delta_o = o
    
    rm_W = self.W
    lambda_W = np.zeros(self.W.shape) 

    rm_U = self.U
    lambda_U = np.zeros(self.U.shape) 

    for t in np.arange(T):

        # Perform forward propagation for one time step
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))

        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)


        delta_o[t, y[t]] -= 1.
        
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))

        # Backpropagation through time (for at most self.bptt_truncate steps)
        # for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
        for bptt_step in np.arange(max(0, t), t+1)[::-1]:
            # print("Backpropagation step t=%d bptt step=%d " % (t, bptt_step))
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        
        self.U -= learning_rate * (dLdU - lambda_U + alpha*(self.U - rm_U) ) 
        self.V -= learning_rate * dLdV 
        self.W -= learning_rate * (dLdW - lambda_W + alpha*(self.W - rm_W) ) 

        # update running estimate -- lambda
        lambda_W -= alpha*(self.W - rm_W)
        lambda_U -= alpha*(self.U - rm_U)

        # update running mean
        rm_W = 0.5*(rm_W + self.W) - (0.5/alpha) * lambda_W
        rm_U = 0.5*(rm_U + self.U) - (0.5/alpha) * lambda_U

RNNNumpy.fptt_training = fptt_training

#### SGD Implementation

Now that we are able to calculate the gradients for our parameters we can implement SGD. I like to do this in two steps: 1. A function `sdg_step` that calculates the gradients and performs the updates for one batch. 2. An outer loop that iterates through the training set and adjusts the learning rate.

In [156]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sdg_step

In [157]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            # model.sgd_step(X_train[i], y_train[i], learning_rate)
            # model.bptt_online_learning(X_train[i], y_train[i], learning_rate)
            model.fptt_training(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1
    return losses

Done! Let's try to get a sense of how long it would take to train our network:

In [158]:
# np.random.seed(10)
# model = RNNNumpy(vocabulary_size)
# %timeit model.sgd_step(X_train[10], y_train[10], 0.005)

In [159]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:100], y_train[:100], nepoch=10, evaluate_loss_after=1)

<ipython-input-151-3f0be9da26f3>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  N = np.sum((len(y_i) for y_i in y))


2023-05-03 13:56:07: Loss after num_examples_seen=0 epoch=0: 8.987504
2023-05-03 13:56:30: Loss after num_examples_seen=100 epoch=1: 8.977822
2023-05-03 13:56:55: Loss after num_examples_seen=200 epoch=2: 8.963784
2023-05-03 13:57:20: Loss after num_examples_seen=300 epoch=3: 8.937948
2023-05-03 13:57:45: Loss after num_examples_seen=400 epoch=4: 8.885456
2023-05-03 13:58:09: Loss after num_examples_seen=500 epoch=5: 8.777311
2023-05-03 13:58:34: Loss after num_examples_seen=600 epoch=6: 8.613676
2023-05-03 13:58:58: Loss after num_examples_seen=700 epoch=7: 8.565171
2023-05-03 13:59:23: Loss after num_examples_seen=800 epoch=8: 8.395709
2023-05-03 13:59:46: Loss after num_examples_seen=900 epoch=9: 6.775044


In [160]:
print(losses)

[(0, 8.98750364771118), (100, 8.97782195821148), (200, 8.963784214916592), (300, 8.937948431605681), (400, 8.88545635000783), (500, 8.777311349963941), (600, 8.613675908682392), (700, 8.565170880223173), (800, 8.395709249790116), (900, 6.775043953667768)]


Good, it seems like our implementation is at least doing something useful and decreasing the loss, just like we wanted.